In [371]:
import hashlib
from bitcoin import base58

# Bitcoin address manipulation  - 
## Why you should check every character in an address when signing a transaction.

**tl;dr**: Generating a valid bitcoin address that shares most of the characters with another address is not difficult if you don't care about having the private key.

At it's most difficult, you can generate a different valid address by changing any 6 non-sequential characters in 10's of hours using this notebook and a standard laptop. If you fix the last 4 characters and up to the first 25 charaters, it it significantly easier (~2-3min to generate a valid malicious address). Fixing the final 6 characters is equivalent to the most difficult case.

**NOTE:** For this example, I will be using this address: `1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ`, it was randomly grabbed from a
block explorer, apologies if it is yours.

## Base58 Basics

Before I explain address serialization, I must first explain base58, and give some examples of converting between bases.

base58 is a base of 58, as opposed to decimal (base 10), hexidecimal (base16), binary (base 2).

There are 58 characters used to represent data that is base58 encoded
(`123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz`)

> In contrast to Base64, the digits of the encoding do not line up well with byte boundaries of the original data. For this reason, the method is well-suited to encode large integers, but not designed to encode longer portions of binary data.

Because 58 is not a power of two, the characters in base58 don't correspond to a fixed number of bytes.
Decimal has the same problem, one byte has 256 values, but that can require 1, 2, or 3 decimal characters.

Because of this, when converting a number to base58 from any other standard base, I don't think you can know what the least significant digits will be as they are dependent on all the digits in the input. I think this is true, but couldn't find a reference.

The most significant digits, however, are not dependendent on all the digits, only the most significant ones.

An example might help explain this:

In [372]:
a_big_number = 1234567890

In [373]:
print("Decimal: ", a_big_number, " Length: ", len(str(a_big_number)))
a_big_hex_number = str(hex(a_big_number))[2:]
print("Hexidecimal: ", a_big_hex_number, " Length: ", len(a_big_hex_number))
a_big_byte_number = a_big_number.to_bytes(4,byteorder='big')
print("Bytes:", a_big_byte_number, " Length: ", len(a_big_byte_number))
a_big_base58_number = base58.encode(a_big_number.to_bytes(4,byteorder='big'))
print("base58: ", a_big_base58_number, " Length: ", len(a_big_base58_number))

Decimal:  1234567890  Length:  10
Hexidecimal:  499602d2  Length:  8
Bytes: b'I\x96\x02\xd2'  Length:  4
base58:  2t6V2H  Length:  6


In [374]:
a_big_number = 1234567891
print("Decimal: ", a_big_number, " Length: ", len(str(a_big_number)))
a_big_hex_number = str(hex(a_big_number))[2:]
print("Hexidecimal: ", a_big_hex_number, " Length: ", len(a_big_hex_number))
a_big_byte_number = a_big_number.to_bytes(4,byteorder='big')
print("Bytes:", a_big_byte_number, " Length: ", len(a_big_byte_number))
a_big_base58_number = base58.encode(a_big_number.to_bytes(4,byteorder='big'))
print("base58: ", a_big_base58_number, " Length: ", len(a_big_base58_number))

Decimal:  1234567891  Length:  10
Hexidecimal:  499602d3  Length:  8
Bytes: b'I\x96\x02\xd3'  Length:  4
base58:  2t6V2J  Length:  6


In [375]:
a_big_number = 1334567890
print("Decimal: ", a_big_number, " Length: ", len(str(a_big_number)))
a_big_hex_number = str(hex(a_big_number))[2:]
print("Hexidecimal: ", a_big_hex_number, " Length: ", len(a_big_hex_number))
a_big_byte_number = a_big_number.to_bytes(4,byteorder='big')
print("Bytes:", a_big_byte_number, " Length: ", len(a_big_byte_number))
a_big_base58_number = base58.encode(a_big_number.to_bytes(4,byteorder='big'))
print("base58: ", a_big_base58_number, " Length: ", len(a_big_base58_number))

Decimal:  1334567890  Length:  10
Hexidecimal:  4f8be3d2  Length:  8
Bytes: b'O\x8b\xe3\xd2'  Length:  4
base58:  32w1YD  Length:  6


Notice that when we change the least significant digit, only the least significant digit changes for the other bases. But if we change a highly significant digit, all the other digits change in the other bases. This is because one decimal digit doesn't map cleanly to a fixed number of digits in any of the other bases. Since bytes and hex do map cleanly, you can change a highly significant digit and not affect the less significant digits in the other base

In [376]:
hex_number = "4f8be3d2"
print(bytes.fromhex(hex_number))
hex_number_2 = "5f8be3d2"
print(bytes.fromhex(hex_number_2))

b'O\x8b\xe3\xd2'
b'_\x8b\xe3\xd2'


## Bitcoin Addresses
Now let's talk about bitcoin addresses. A bitcoin address has three components: 
- The version byte
- The data
- The Checksum

### The Version Byte
> 0x00 for P2PKH addresses on the main Bitcoin network (mainnet)  
> 0x6f for P2PKH addresses on the Bitcoin testing network (testnet)  
> 0x05 for P2SH addresses on mainnet  
> 0xc4 for P2SH addresses on testnet  



### The Data
For a P2PKH address, the data portion of an address is the hash (actually hashed twice: RIPEMD(SHA256(pubkey))) of a public key

### The Checksum
Bitcoin Addresses are actually Base58Check encoded, not simply base58. In Base58Check, you first double-hash the version byte and data with sha256. You take the first 4 bytes of the resulting hash and append it to the data, and then you convert the full (version byte + data + checksum) byte array into base58.

### A Note on Checksum collisions

By only taking the first 4 bytes of the hash, the checksum field is far from immune to collisions. SHA256 properly generates any even distribution of outputs. 4 bytes can only encode $256^4$ values (4,294,967,296), so if you want to find a hash collision, you onyl need to try an expected 4.3 Billion values per collision. This is important for this discussion, but it isn't the end of the story.

You might think at this point, "Cool, so if I want to create a malicious bitcoin address, I'll just decode it to bytes, and start changing bytes until I find a checksum collision, and I'm set!" But you'd be wrong. If you skimmed the base58 discussion above, maybe take a closer read. If you change some bytes in the middle of an address, when you go to encode it in base58, all of the lower significant digits are modified in base58.

So now you might think: "Shit, so I have to modify the data, find a checksum collision, and then hope it encodes to the same last 4 digits? I'm screwed, bitcoin addresses are hard."

It's true, bitcoin addresses are confusing, but no, it's not that difficult to create a malicious address, and that's because while the checksum has to be *valid*, it doesn't have to be the *same* checksum of the original address.

### Generating a Malicious address.

Let's start simple. Let's say you want a valid bitcoin address with the same $A$ number of characters at the beginning, and the last digit to be the same. Let's assume that the resulting addresses will have an even distibution of final characters, since we're going to be changing a bunch of digits (some in the data, and all of the checksum).

If it's an even distribution, for each value we try, we have a 1-in-58 probability that it will be the correct value.
Let's give it a try!

We'll try every value in one byte, that's 256 values, so we should expect to see 256/58 = 4.4 matches

In [377]:
def cycle_through_one_byte(ver, data, digit, match):
    dataarray = bytearray(data)
    for i in range(256):
        # Change a byte
        dataarray[digit] = i
        # Compute the double sha256 hash and take the first 4 bytes
        check1 = bitcoin.core.Hash(ver + dataarray)[:4]
        # convert to base58
        addr = base58.encode(ver+dataarray+check1)
        # CHeck the last digit
        if (addr[-1:] == match):
            print('found: ', addr)

In [378]:
original_address ='1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ'
k = base58.decode(original_address)
verbyte, data, check0 = k[0:1], k[1:-4], k[-4:]

In [379]:
cycle_through_one_byte(verbyte, data, 19, 'Z')

found:  1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFxYGxT7Z
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFyeGfzQZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cG438ERVZ


In [380]:
cycle_through_one_byte(verbyte, data, 19,  '1')

found:  1HU1LDBXUg73f2ro2e2dB3XY8cFkFSmCQ1
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFwPyAZ91
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFysN4xF1
found:  1HU1LDBXUg73f2ro2e2dB3XY8cG4pxmyY1
found:  1HU1LDBXUg73f2ro2e2dB3XY8cGB5Rhk51


In [381]:
%time cycle_through_one_byte(verbyte, data, 19, '2')

found:  1HU1LDBXUg73f2ro2e2dB3XY8cFnB2tYx2
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFqTayv52
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFzcYYU22
found:  1HU1LDBXUg73f2ro2e2dB3XY8cG2q3M6u2
found:  1HU1LDBXUg73f2ro2e2dB3XY8cG4RpLuB2
CPU times: user 5.46 ms, sys: 113 µs, total: 5.57 ms
Wall time: 5.47 ms


Looks pretty solid. I think this is dependent on which digit you're changing, though I don't know how to generalize at this time. For example, if you change the 10th byte instead of the 19th:

In [382]:
cycle_through_one_byte(verbyte, data, 10, 'Z')

found:  1HU1LDBXUg73f2rS5EwnM2GQLJizdsKtzZ
found:  1HU1LDBXUg73f2rZo25fuygJ3nYycfXiyZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ
found:  1HU1LDBXUg73f2rqEaMT3tW5UkCwaKM2oZ
found:  1HU1LDBXUg73f2rw7jtKPUSBjn5Jr7ykzZ
found:  1HU1LDBXUg73f2rzReNZCjtzkz11r3GvJZ
found:  1HU1LDBXUg73f2rzny6MrDPkefzYDh1k1Z
found:  1HU1LDBXUg73f2s73TM2qGpcoPrRqdNtSZ
found:  1HU1LDBXUg73f2sKYRrNoPgM6qaDAqb3MZ
found:  1HU1LDBXUg73f2sf2Y413WxTLP8vqTGk6Z


In [383]:
cycle_through_one_byte(verbyte, data, 10, '1')

found:  1HU1LDBXUg73f2sgs9f2GtSEnq6YQ4sgH1


Though maybe that's just statistics being fun.

Let's move on to 2 digit matches. With 2 digits we have $58^2$ so 3,364 options, But if we cycle through all values in 2 bytes, that's $256^2$ or 65,536. SO we should expect to see, 19.5 matches

In [384]:
def cycle_through_two_bytes(ver, data, match):
    total_matches = 0
    dataarray = bytearray(data)
    for i in range(256):
        # Change a byte
        dataarray[19] = i
        for j in range(256):
            dataarray[18] = j
            # Compute the double sha256 hash and take the first 4 bytes
            check1 = bitcoin.core.Hash(ver + dataarray)[:4]
            # convert to base58
            addr = base58.encode(ver+dataarray+check1)
            # CHeck the last digit
            if (addr[-2:] == match):
                total_matches+=1
                print('found: ', addr)
    print("Total Matches:", total_matches)

In [385]:
cycle_through_two_bytes(verbyte, data, 'ZZ')

found:  1HU1LDBXUg73f2ro2e2dB3XY8bhL6FNdZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8b5VnZNvZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cY9otddZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cQhSD81ZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8aq5LtioZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8ay3kBjzZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8an97xSxZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cXKEfctZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bd2iMNCZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8b99mNuQZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bt1pxGTZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8c7zDqXXZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cBVPbUQZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8b1nWCCgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cSTME4jZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bp7Nc5dZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8c549A7UZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bxasQpoZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bRE69KdZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bVDxYCzZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8agwJoHTZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8aZWPiFsZZ
found:  1HU

In [386]:
cycle_through_two_bytes(verbyte, data, 'aZ')

found:  1HU1LDBXUg73f2ro2e2dB3XY8ae7QdPKaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bdPxVDTaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bJzuAiZaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8b99L4dDaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bb3swQDaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bm3A3i8aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bSekXz2aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cAVywr3aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bLif2joaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bAmk3KXaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8beCoj1JaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8aZy1QL2aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bwAkqDYaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bmiDSWGaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8c1CFZ9waZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bhH2ykPaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8ayy1yF9aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bPtonGxaZ
Total Matches: 18


In [387]:
%time cycle_through_two_bytes(verbyte, data, '11')

found:  1HU1LDBXUg73f2ro2e2dB3XY8cQfuTxL11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bvp6pgm11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bkN6vHT11
found:  1HU1LDBXUg73f2ro2e2dB3XY8az3nv1o11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bMVM9AF11
found:  1HU1LDBXUg73f2ro2e2dB3XY8c1rC2wg11
found:  1HU1LDBXUg73f2ro2e2dB3XY8btWHoTC11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cWrnG9d11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bR8RfEb11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bEgqkAu11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cht15Xm11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cGVVvF811
found:  1HU1LDBXUg73f2ro2e2dB3XY8agtFyKQ11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bjfNS2s11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bPkcyca11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bGniJ6M11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cM4y1rG11
found:  1HU1LDBXUg73f2ro2e2dB3XY8biGTxzX11
found:  1HU1LDBXUg73f2ro2e2dB3XY8akWcvRn11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cXd3wQH11
Total Matches: 20
CPU times: user 1.16 s, sys: 10.3 ms, total: 1.17 s
Wall time: 1.17 s


Pretty solid so far. Notice that I'm changing only the least significant byte in the data section. This means the whole first section of address is unchanged.

Let's move on to 3 digits.

With 3 digits we have $58^3$ so 195,112 options, But if we cycle through all values in 3 bytes, that's $256^3$ or 16,777,216. SO we should expect to see, 86 matches

In [388]:
def cycle_through_three_bytes(ver, data, match):
    total_matches = 0
    dataarray = bytearray(data)
    for i in range(256):
        # Change a byte
        dataarray[19] = i
        for j in range(256):
            dataarray[18] = j
            for k in range(256):
                dataarray[17] = k
                # Compute the double sha256 hash and take the first 4 bytes
                check1 = bitcoin.core.Hash(ver + dataarray)[:4]
                # convert to base58
                addr = base58.encode(ver+dataarray+check1)
                # CHeck the last digit
                if (addr[-3:] == match):
                    total_matches+=1
                    #print('found: ', addr)
    print("Total Matches:", total_matches)

In [389]:
print(original_address)

1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ


In [390]:
%time cycle_through_three_bytes(verbyte, data, 'gZZ')

Total Matches: 79
CPU times: user 4min 20s, sys: 377 ms, total: 4min 21s
Wall time: 4min 21s


Along with agreeing with our statistics so far, notice that the execution time has grown linearly with number of operations. 5ms for one byte, 1.2s for two bytes (256x the ops of one byte), 261s for three bytes (256x the ops of two bytes)

Let's move on to 4 digits. Which for some reason is a standard, "check the first and last 4". This is probably due to misunderstanding the checksum field. 4 base58 digits can only encode 3 bytes worth of data $(58^4 < 256^3)$, and as mentioned earlier, you can't decode the least significant digits only, you need the full value to decode.

With 4 digits we have $58^4$ so 11,316,496 options, But if we can cycle through all values in still only 3 bytes because that's $256^3$ or 16,777,216. So we should expect to see, 1.5 matches.

We can iterate another byte a 4 times to boost the expect matches to 6 while expecting ~20min execution time.

In [391]:
def cycle_through_four_bytes(ver, data, match):
    total_matches = 0
    dataarray = bytearray(data)
    for i in range(4):
        # Change a byte
        dataarray[19] = i
        for j in range(256):
            dataarray[18] = j
            for k in range(256):
                dataarray[17] = k
                for l in range(256):
                    dataarray[16] = l
                    # Compute the double sha256 hash and take the first 4 bytes
                    check1 = bitcoin.core.Hash(ver + dataarray)[:4]
                    # convert to base58
                    addr = base58.encode(ver+dataarray+check1)
                    # CHeck the last digit
                    if (addr[-4:] == match):
                        total_matches+=1
                        print('found: ', addr)
    print("Total Matches:", total_matches)

In [359]:
print(original_address)

1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ


In [360]:
%time cycle_through_four_bytes(verbyte, data, 'FgZZ')

found:  1HU1LDBXUg73f2ro2e2dB3YAZkLzhmFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3YAsH6C6bFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XeYUj3vkFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3Xx2JDVAbFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3Y6ZfhPG1FgZZ
found:  1HU1LDBXUg73f2ro2e2dB3Y5QbkmDLFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3YEXcH33JFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XhYFrPERFgZZ
Total Matches: 8
CPU times: user 17min 16s, sys: 913 ms, total: 17min 16s
Wall time: 17min 17s


Great. So we can create duplicate valid first4...last4 bitcoin addresses at a rate of once every 2 minutes.

Now what about fixing the last 5 digits? 

Now we have $58^5$ = 656,356,768, which is 40x more than 3 bytes, but still well less than the full $256^4$. That'll be roughly 3 hours of execution to generate one, we'll leave that as an exercise. 

And what about 6 or 7 digits?

6 digits gives us a 1 in 38,068,692,544 chance to find a valid address. That's interesting because now we're above $256^4$ which is the hash-collision scale. At this point, I think it will be faster to invert this process.

Instead of searching for matches to our base58 pattern, let's search through base58 values to find valid addresses.
A given base58 string should have a 1-in-4,294,967,296 chance of being valid $(256^4)$.

Based on our earlier benchmarks, that should take roughly 20 hours of execution. To get to that number of options, we would need to be able to change 6 digits of the address, and I don't think they need to be contiguous.

I'll try this in the future and update.